In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

In [2]:
# 맥북 
data = pd.read_csv('/Users/dongyokim/Library/Mobile Documents/com~apple~CloudDocs/2022:1/마케팅 애널리틱스/중간 과제/직방 competition data 4/train.csv')

In [3]:
# 윈도우 
data = pd.read_csv('C:/Users/Innovation/Desktop/김동요/jupyter/Data/MA/직방 competition data 4/직방 competition data 4/train.csv')

In [4]:
data.head()

,key,apartment_id,city,transaction_year_month,transaction_date,year_of_completion,exclusive_use_area,floor,latitude,longitude,...,lowest_building_in_sites,heat_type,heat_fuel,room_id,supply_area,total_household_count_of_area_type,room_count,bathroom_count,front_door_structure,transaction_real_price
0,0,5584,1,200601,11~20,1999,47.43,6,37.585965,127.000231,...,4.0,individual,gas,91120,65.63,46,1.0,1.0,corridor,215000000
1,1,5584,1,200601,11~20,1999,44.37,8,37.585965,127.000231,...,4.0,individual,gas,91119,61.39,10,2.0,1.0,corridor,200000000
2,2,5059,1,200601,11~20,1992,54.70,8,37.580511,127.014016,...,9.0,individual,gas,8430,72.36,201,2.0,1.0,corridor,168000000
3,3,2816,1,200601,11~20,1993,64.66,11,37.580324,127.011788,...,11.0,individual,gas,5839,87.30,284,2.0,1.0,corridor,165000000
4,4,2816,1,200601,11~20,1993,106.62,7,37.580324,127.011788,...,11.0,individual,gas,5836,127.74,112,4.0,2.0,stairway,280000000


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1601458 entries, 0 to 1601457
Data columns (total 25 columns):
 #   Column                              Non-Null Count    Dtype  
---  ------                              --------------    -----  
 0   key                                 1601458 non-null  int64  
 1   apartment_id                        1601458 non-null  int64  
 2   city                                1601458 non-null  int64  
 3   transaction_year_month              1601458 non-null  int64  
 4   transaction_date                    1601458 non-null  object 
 5   year_of_completion                  1601458 non-null  int64  
 6   exclusive_use_area                  1601458 non-null  float64
 7   floor                               1601458 non-null  int64  
 8   latitude                            1601458 non-null  float64
 9   longitude                           1601458 non-null  float64
 10  address_by_law                      1601458 non-null  int64  
 11  total_parki

In [7]:
# EDA에 추가 
# sns.distplot(data['transaction_real_price'])
# plt.title('Original Sale Price Histogram')
# plt.show()

In [8]:
# EDA에 추가 
# plt.title('Log Transformed Sale Price Histogram')
# log_SalePrice = np.log1p(data['transaction_real_price'])
# sns.distplot(log_SalePrice)
# plt.show()

In [9]:
# EDA에 추가 & 다중공선성 파악  
# plt.figure(figsize=(13, 13))
# sns.heatmap(data.corr(), annot=True)

## 새로운 열 추가 

In [59]:
# 해당 평형의 방을 이용하는 세대수가  전체 입주 세대수에서 얼마나 차지하는지 나타냄 
household_ratio = data.total_household_count_of_area_type / data.total_household_count_in_sites 
data['해당 평형 이용 세대수 비율'] = household_ratio

# 해당 방의 평수를 나타냄 
feet_3025 = round(data.exclusive_use_area * 0.3025, 4)
data['방 평수'] = feet_3025

# 평당 가격 
price_3025 = round(data.transaction_real_price / feet_3025, 4)
data['평당 가격'] = price_3025

# 공급면적(전용면적 + 주거공용면적)의 평수 
supply_3025 = round(data.supply_area * 0.3025, 4)
data['공급 면적 평수'] = supply_3025

# 계약 년 & 월 
transaction_year = data.transaction_year_month.apply(lambda x: str(x)[0:4])
data['transcation_year'] = transaction_year + '년'
transaction_month = data.transaction_year_month.apply(lambda x: str(x)[4:6])
data['transcation_month'] = transaction_month + '월'

transaction_date = transaction_year + "-" + transaction_month + "-01"
transaction_date = transaction_date.astype('datetime64')
data['transaction_date'] = transaction_date

# 계약연도와 아파트 준공년도의 차이(아파트 나이)
apartment_age = data.transaction_year_month.apply(lambda x: int(str(x)[0:4])) - data.year_of_completion + 1
data['거래연도 기준 아파트 나이'] = apartment_age

# 아파트 층 수 높이 순위(최고층 - 해당 방의 층 + 1)
floor_rank = data.tallest_building_in_sites - data.floor + 1
data['층 순위'] = floor_rank


In [61]:
data.corr()['transaction_real_price'].sort_values(ascending=False)

transaction_real_price                1.000000
평당 가격                                 0.825508
공급 면적 평수                              0.521845
supply_area                           0.521845
방 평수                                  0.518922
exclusive_use_area                    0.518922
city                                  0.399349
latitude                              0.386292
room_count                            0.383066
bathroom_count                        0.331598
total_parking_capacity_in_site        0.310130
key                                   0.296135
apartment_building_count_in_sites     0.290689
transaction_year_month                0.285657
total_household_count_in_sites        0.216658
tallest_building_in_sites             0.193310
apartment_id                          0.160393
year_of_completion                    0.132442
floor                                 0.129241
lowest_building_in_sites              0.113520
층 순위                                  0.097949
거래연도 기준 아파트 나

In [63]:
data.corr()['평당 가격'].sort_values(ascending=False)

평당 가격                                 1.000000
transaction_real_price                0.825508
city                                  0.546510
latitude                              0.532850
apartment_building_count_in_sites     0.398329
key                                   0.348511
transaction_year_month                0.335251
total_parking_capacity_in_site        0.321249
total_household_count_in_sites        0.307601
apartment_id                          0.105852
거래연도 기준 아파트 나이                        0.089271
공급 면적 평수                              0.083655
supply_area                           0.083655
방 평수                                  0.073694
exclusive_use_area                    0.073694
bathroom_count                        0.073111
tallest_building_in_sites             0.067870
year_of_completion                    0.058277
floor                                 0.052648
room_count                            0.047320
total_household_count_of_area_type    0.046956
층 순위         

In [64]:
print('데이터 세트의 Shape:', data.shape)
print('\n전체 피처의 type: \n', data.dtypes.value_counts())
isnull_series = data.isnull().sum()
print('\nNull 칼럼과 그 건수:\n ', isnull_series[isnull_series > 0].sort_values(ascending=False))

데이터 세트의 Shape: (1601458, 33)

전체 피처의 type: 
 float64           14
int64             13
object             5
datetime64[ns]     1
dtype: int64

Null 칼럼과 그 건수:
  total_parking_capacity_in_site    91813
front_door_structure              13892
heat_fuel                          9667
heat_type                          2017
room_count                          691
bathroom_count                      691
tallest_building_in_sites             9
lowest_building_in_sites              9
층 순위                                  9
dtype: int64


In [65]:
data.columns

Index(['key', 'apartment_id', 'city', 'transaction_year_month',
       'transaction_date', 'year_of_completion', 'exclusive_use_area', 'floor',
       'latitude', 'longitude', 'address_by_law',
       'total_parking_capacity_in_site', 'total_household_count_in_sites',
       'apartment_building_count_in_sites', 'tallest_building_in_sites',
       'lowest_building_in_sites', 'heat_type', 'heat_fuel', 'room_id',
       'supply_area', 'total_household_count_of_area_type', 'room_count',
       'bathroom_count', 'front_door_structure', 'transaction_real_price',
       '해당 평형 이용 세대수 비율', '방 평수', '평당 가격', '공급 면적 평수', 'transcation_year',
       'transcation_month', '거래연도 기준 아파트 나이', '층 순위'],
      dtype='object')

## 데이터 셋 설정 및 구분 

In [67]:
# EDA & 태블로 시각화용 데이터 

# null이 너무 많은 컬럼과 불필요한 컬럼 삭제
tru_data = data.drop(['key', 'apartment_id', 'transaction_year_month', 'room_id', 'front_door_structure','heat_type', 'heat_fuel'], axis=1)

tru_data.dropna(how='any', inplace=True)

In [68]:
print('데이터 세트의 Shape:', tru_data.shape)
print('\n전체 피처의 type: \n', tru_data.dtypes.value_counts())
isnull_series = tru_data.isnull().sum()
print('\nNull 칼럼과 그 건수:\n ', isnull_series[isnull_series > 0].sort_values(ascending=False))

데이터 세트의 Shape: (1508954, 26)

전체 피처의 type: 
 float64           14
int64              9
object             2
datetime64[ns]     1
dtype: int64

Null 칼럼과 그 건수:
  Series([], dtype: int64)


In [72]:
tru_data.set_index(inplace=True)

TypeError: set_index() missing 1 required positional argument: 'keys'

In [70]:
# 데이터 csv파일로 추출 
tru_data.to_csv('EDA_dataset.csv')

In [ ]:
# 회귀분석용 데이터 

# null이 너무 많은 컬럼과 불필요한 컬럼 삭제
tru_data = data.drop(['key', 'apartment_id', 'transaction_year_month', 'room_id', 'front_door_structure','heat_type', 'heat_fuel'], axis=1)

tru_data.dropna(how='any', inplace=True)

In [71]:
tru_data.city.value_counts()

1    957474
0    551480
Name: city, dtype: int64

In [85]:
#  'transaction_real_price' 로그 변환 
# original_SalePrice = data['transaction_real_price']
# data['transaction_real_price'] = np.log1p(data['transaction_real_price'])

# Null이 너무 많은 컬럼과 불필요한 컬럼 삭제
tru_data = data.drop(['key', 'apartment_id', 'address_by_law', 'longitude', 'latitude', 'total_parking_capacity_in_site', 'front_door_structure','heat_type', 'heat_fuel', 'transaction_year_month', 'year_of_completion'], axis=1)

# 드롭하지 않은 숫자형 Null 칼럼은 평균값으로 대체 
data.dropna(how='any', inplace=True)



In [88]:
data.exclusive_use_area = data.exclusive_use_area * 0.3025
data.supply_area = data.supply_area * 0.3025

In [89]:
# 평당 거래가를 구하기 위해 종속변수를 전체 평으로 나눠줌 
data.transaction_real_price = (data.transaction_real_price / data.exclusive_use_area)

In [90]:
data.head()

,city,exclusive_use_area,floor,total_household_count_in_sites,apartment_building_count_in_sites,tallest_building_in_sites,lowest_building_in_sites,room_id,supply_area,total_household_count_of_area_type,room_count,bathroom_count,transaction_real_price
0,1,14.347575,6,136,1,8.0,4.0,91120,19.853075,46,1.0,1.0,1.498511e+07
1,1,13.421925,8,136,1,8.0,4.0,91119,18.570475,10,2.0,1.0,1.490099e+07
2,1,16.546750,8,585,5,14.0,9.0,8430,21.888900,201,2.0,1.0,1.015305e+07
3,1,19.559650,11,919,7,15.0,11.0,5839,26.408250,284,2.0,1.0,8.435734e+06
4,1,32.252550,7,919,7,15.0,11.0,5836,38.641350,112,4.0,2.0,8.681484e+06


In [92]:
data.corr()['transaction_real_price'].sort_values(ascending=False)

transaction_real_price                1.000000
city                                  0.546481
apartment_building_count_in_sites     0.398447
total_household_count_in_sites        0.307655
supply_area                           0.083725
exclusive_use_area                    0.073812
bathroom_count                        0.073110
tallest_building_in_sites             0.067774
floor                                 0.052553
room_count                            0.047320
total_household_count_of_area_type    0.046928
lowest_building_in_sites              0.006676
room_id                              -0.136317
Name: transaction_real_price, dtype: float64

In [75]:
data.corr()['transaction_real_price'].sort_values(ascending=False)

transaction_real_price                1.000000
supply_area                           0.521792
exclusive_use_area                    0.518895
city                                  0.399426
room_count                            0.383066
bathroom_count                        0.331597
apartment_building_count_in_sites     0.290652
total_household_count_in_sites        0.216589
tallest_building_in_sites             0.193204
floor                                 0.129192
lowest_building_in_sites              0.113397
total_household_count_of_area_type   -0.030709
room_id                              -0.071414
Name: transaction_real_price, dtype: float64

In [37]:
print('get_dummies() 수행 전 데이터 Shape:', data.shape)
data_ohe = pd.get_dummies(data)
print('get_dummies() 수행 후 데이터 Shape: ', data_ohe.shape)

null_column_count = data_ohe.isnull().sum()[data_ohe.isnull().sum() > 0]
print('## Null 피처의 Type : \n', data_ohe.dtypes[null_column_count.index])

get_dummies() 수행 전 데이터 Shape: (1600758, 15)
get_dummies() 수행 후 데이터 Shape:  (1600758, 20)
## Null 피처의 Type : 
 Series([], dtype: object)


In [38]:
data_ohe

,city,transaction_year_month,year_of_completion,exclusive_use_area,floor,total_household_count_in_sites,apartment_building_count_in_sites,tallest_building_in_sites,lowest_building_in_sites,supply_area,total_household_count_of_area_type,room_count,bathroom_count,transaction_real_price,transaction_date_11~20,transaction_date_1~10,transaction_date_21~28,transaction_date_21~29,transaction_date_21~30,transaction_date_21~31
0,1,200601,1999,47.4300,6,136,1,8.0,4.0,65.63,46,1.0,1.0,215000000,1,0,0,0,0,0
1,1,200601,1999,44.3700,8,136,1,8.0,4.0,61.39,10,2.0,1.0,200000000,1,0,0,0,0,0
2,1,200601,1992,54.7000,8,585,5,14.0,9.0,72.36,201,2.0,1.0,168000000,1,0,0,0,0,0
3,1,200601,1993,64.6600,11,919,7,15.0,11.0,87.30,284,2.0,1.0,165000000,1,0,0,0,0,0
4,1,200601,1993,106.6200,7,919,7,15.0,11.0,127.74,112,4.0,2.0,280000000,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1601453,0,201810,1999,118.4700,14,819,8,27.0,13.0,143.45,108,4.0,2.0,427000000,0,0,0,0,0,1
1601454,1,201810,2007,59.9900,4,1122,22,15.0,8.0,79.98,254,3.0,2.0,771000000,0,0,0,0,0,1
1601455,0,201810,2014,84.9669,31,1326,9,35.0,27.0,109.77,209,3.0,2.0,343000000,0,0,0,0,0,1
1601456,1,201810,1996,59.3400,4,107,1,19.0,11.0,88.37,4,3.0,1.0,485000000,0,0,0,0,0,1
